# Informations sur les fournisserus de soins

Données
* Base de personnels de soin: `ps-infospratiques.csv` (annuaire de la CNAM) [https://www.data.gouv.fr/fr/datasets/annuaire-sante-de-la-cnam/]
* Base de correspondance entre CP et code commune INSEE [https://www.data.gouv.fr/fr/datasets/r/554590ab-ae62-40ac-8353-ee75162c05ee]
 


In [1]:
import pandas as pd
import numpy as np

In [2]:
import os.path
outdir="../data"

In [3]:
data = pd.read_csv("../data/ps-infospratiques.csv", sep=";", header=None, encoding="latin_1")
data.rename(columns={
                0:"Sexe", 
                1:"nom", 
                2:"prenom", 
                3:"Adresse ligne 1,", 
                4:"Adresse ligne 2",
                5:"Adresse ligne 3",
                6:"Adresse ligne 4",
                7:"CP", ## un numéro qui sembl être un CP (mais des soucis avec les 2 ... (?))
                8:"Ville",
                9:"Téléphone", 
                10:"Profession", #code de profession (voir nommenclature ci-dessous)
                11:"mode", #mode d'exercice
                12:"nature", #nature d'exercice
                13:"convention", 
                14:"option contrat", 
                15:"sesam_vital", 
                16:"Type", # Liberal/hors activité, etc.
                17:"Type consultation", 
                18:"heure_debut", 
                19:"heure_debut",
                20:"jour"}, inplace=True)

/usr/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3062: DtypeWarning: Columns (16,17,18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
#data = pd.read_csv("../data/ps-infospratiques.csv", sep=";", header=None, encoding="latin_1", usecols=[0,3,4,5,6,7,8,10,11,12,16])
#data.rename(columns={
#                0:"Sexe",  
#                3:"Adresse ligne 1,", 
#                4:"Adresse ligne 2",
#                5:"Adresse ligne 3",
#                6:"Adresse ligne 4",
#                7:"CP", ## un numéro qui sembl être un CP (mais des soucis avec les 2 ... (?))
#                8:"Ville",
#                10:"Profession", #code de profession (voir nommenclature ci-dessous)
#                11:"mode", #mode d'exercice
#                12:"nature", #nature d'exercice
#                16:"Type", # Liberal/hors activité, etc.
#            }, inplace=True)

In [5]:
data.head()

,Sexe,nom,prenom,"Adresse ligne 1,",Adresse ligne 2,Adresse ligne 3,Adresse ligne 4,CP,Ville,Téléphone,...,mode,nature,convention,option contrat,sesam_vital,Type,Type consultation,heure_debut,heure_debut,jour
0,F,CUGNIET,ODILE,NaN,NaN,35 RUE CENTRALE,NaN,1230,TENAY,NaN,...,NaN,3,c1,N,O,NaN,NaN,NaN,NaN,NaN
1,H,TANAZACQ,REMI,NaN,NaN,1 ROUTE DE CHAMOISE,NaN,1430,ST MARTIN DU FRENE,NaN,...,NaN,3,c1,O,O,NaN,NaN,NaN,NaN,NaN
2,H,HOJABR,AMIR THEODORE,NaN,NaN,76 RUE GRANDE,NaN,1220,DIVONNE LES BAINS,971169655.0,...,NaN,3,c3,N,O,NaN,NaN,NaN,NaN,NaN
3,H,HOJABR,AMIR THEODORE,NaN,CLINIQUE CENDANEG,180 ROUTE DU NANT,NaN,1280,PREVESSIN MOENS,NaN,...,NaN,3,c3,N,O,NaN,NaN,NaN,NaN,NaN
4,F,ADAM,EMILIE,NaN,NaN,180 ROUTE DU NANT,NaN,1280,PREVESSIN MOENS,NaN,...,NaN,3,c3,N,O,NaN,NaN,NaN,NaN,NaN


In [6]:
# Médecins libéraux exercant en ville
medecins=data[(data['Type']!=0) & (data['Type']!=6) & (data['Type']!=7) & (data['Type']!=8) ]

# codage du sex selon la IR_SEX_COD
medecins["Sexe"].fillna("9",inplace=True)
medecins["Sexe"]=medecins["Sexe"].str.replace(pat="F",repl="2")
medecins["Sexe"]=medecins["Sexe"].str.replace(pat="H",repl="1")
# suppression des CEDEX
medecins["Ville"]=medecins["Ville"].str.replace(pat=" CEDEX.*",repl="", regex=True)

medecins=medecins[["Sexe","nom","prenom","CP","Ville","Profession"]].drop_duplicates()
# on supprime les "non-médecins" (parce qu'ils ne correspondent pas à cette classe des spécialistes)
medecins=medecins[ medecins["Profession"]!=1 ] # suppression dans ambulanciers
medecins=medecins[ medecins["Profession"]!=2 ] # suppression dans anathamo-patho
medecins=medecins[ (medecins["Profession"]<18) | (medecins["Profession"]>21) ] # suppression des dentistes
medecins=medecins[ (medecins["Profession"]<24) | (medecins["Profession"]>32) ] # suppression des fournisseurs de matériel
medecins=medecins[ (medecins["Profession"]!=39) ] # suppression des infirmiers
medecins=medecins[ (medecins["Profession"]<40) | (medecins["Profession"]>42)] # suppression des laboratoires
medecins=medecins[ (medecins["Profession"]!=43) ] # suppression des kinés
medecins=medecins[ (medecins["Profession"]!=57) ] # suppression des orthophonistes
medecins=medecins[ (medecins["Profession"]!=58) ] # suppression des orthoptiste
medecins=medecins[ (medecins["Profession"]!=58) ] # suppression des pédicures-podologues
medecins=medecins[ (medecins["Profession"]<62) | (medecins["Profession"]>63)] # suppression des pharmaciens
medecins=medecins[ (medecins["Profession"]!=71) ] # suppression des sages-femmes

medecins.drop(["nom","prenom"],axis=1, inplace=True)
medecins

/usr/lib64/python3.8/site-packages/pandas/core/generic.py:6287: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


,Sexe,CP,Ville,Profession
0,2,1230,TENAY,45
1,1,1430,ST MARTIN DU FRENE,6
2,1,1220,DIVONNE LES BAINS,56
3,1,1280,PREVESSIN MOENS,56
4,2,1280,PREVESSIN MOENS,15
...,...,...,...,...
1176601,2,75015,PARIS,65
1176607,2,13007,MARSEILLE,67
1176608,1,40230,ST VINCENT DE TYROSSE,6
1176614,1,89480,COULANGES SUR YONNE,6


In [7]:
medecins.head()

,Sexe,CP,Ville,Profession
0,2,1230,TENAY,45
1,1,1430,ST MARTIN DU FRENE,6
2,1,1220,DIVONNE LES BAINS,56
3,1,1280,PREVESSIN MOENS,56
4,2,1280,PREVESSIN MOENS,15


## Get code commune Insee

In [8]:
data = pd.read_csv("../data/laposte_hexasmal.csv", sep=";", usecols=["Code_commune_INSEE","Code_postal","Nom_commune"])

In [9]:
# du fait des libellés d'acheminement, il peut y avoir plusieurs entrées identique pour la correspondance CP/INSEE
data.drop_duplicates(inplace=True)
data

,Code_commune_INSEE,Nom_commune,Code_postal
0,01006,AMBLEON,1300
1,01007,AMBRONAY,1500
2,01014,ARBENT,1100
3,01015,ARBOYS EN BUGEY,1300
4,01016,ARBIGNY,1190
...,...,...,...
39187,98822,POINDIMIE,98822
39188,98829,THIO,98829
39189,98831,VOH,98833
39190,98832,YATE,98834


In [10]:
medecins_insee=pd.merge(data, medecins, how="right", right_on=["CP","Ville"], left_on=["Code_postal","Nom_commune"])[["Code_commune_INSEE","Ville","CP","Sexe","Profession"]]
medecins_insee.head()

,Code_commune_INSEE,Ville,CP,Sexe,Profession
0,01007,AMBRONAY,1500,2,45
1,01007,AMBRONAY,1500,1,45
2,01007,AMBRONAY,1500,1,45
3,01014,ARBENT,1100,1,45
4,01014,ARBENT,1100,2,61


In [11]:
##même taille que la liste initial des médecins ...
print(len(medecins)) 
print(len(medecins_insee)) 

160409
160409


In [12]:
medecins_insee[medecins_insee["Code_commune_INSEE"].fillna("")!=""].to_csv( os.path.join(outdir,"medecins.csv") )

In [12]:
#mais ... des correspondances n'ont pas fontionner
medecins_manquants=medecins_insee[medecins_insee["Code_commune_INSEE"].fillna("")==""]
medecins_manquants

,Code_commune_INSEE,Ville,CP,Sexe,Profession
126318,NaN,ST MARTIN DU FRESNE,1430,1,6
126319,NaN,ST MARTIN DU FRESNE,1430,1,45
126320,NaN,ST MARTIN DU FRESNE,1430,1,52
126321,NaN,ST MARTIN DU FRESNE,1430,2,45
126322,NaN,ST MARTIN DU FRESNE,1430,1,6
...,...,...,...,...,...
160404,NaN,SEVRIER,74330,2,56
160405,NaN,ST AUBIN DE BAUBIGNE,79700,1,45
160406,NaN,HYERES,83407,1,67
160407,NaN,CAPESTERRE B EAU,97130,2,45


Problemes dans les codages de la base des médecins a priori
* probleme dans certains noms de communes (FRESNE a la place de FRENE par exemple)
* probleme également dans ces CP

La solution ... est d'améliorer le matching des communes dans un premier temps

In [16]:
#utilisation d'une librairie permettant de faire une matching approximatif
import difflib

# Pour chaque ville, on cherche la correspondance la plus proche dans la base des communes disponibles !
## ATTENTION: cette étape est longue en calcul
def proceed(x):
    corresp = difflib.get_close_matches(x, data["Nom_commune"])
    if len(corresp)==0:
        return ""
    else:
        return corresp[0]
    return 

#medecins_manquants['ville_align']=medecins_manquants["Ville"].apply(proceed)

<ipython-input-16-d82dad159e96>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  medecins_manquants['ville_align']=medecins_manquants["Ville"].apply(proceed)


In [17]:
medecins_manquants

,Code_commune_INSEE,Ville,CP,Sexe,Profession,ville_align
126318,NaN,ST MARTIN DU FRESNE,1430,1,6,ST MARTIN DU FRENE
126319,NaN,ST MARTIN DU FRESNE,1430,1,45,ST MARTIN DU FRENE
126320,NaN,ST MARTIN DU FRESNE,1430,1,52,ST MARTIN DU FRENE
126321,NaN,ST MARTIN DU FRESNE,1430,2,45,ST MARTIN DU FRENE
126322,NaN,ST MARTIN DU FRESNE,1430,1,6,ST MARTIN DU FRENE
...,...,...,...,...,...,...
160404,NaN,SEVRIER,74330,2,56,SEVRIER
160405,NaN,ST AUBIN DE BAUBIGNE,79700,1,45,ST AUBIN D AUBIGNE
160406,NaN,HYERES,83407,1,67,HYERES
160407,NaN,CAPESTERRE B EAU,97130,2,45,CAPESTERRE BELLE EAU


Ensuite, on peut recommencer à faire l'alignement, mais sur la ville normalisée ....

In [18]:
medecins_insee=pd.merge(data, medecins, how="right", right_on=["CP","ville_align"], left_on=["Code_postal","Nom_commune"])[["Code_commune_INSEE","Ville","CP","Sexe","Profession"]]
medecins_insee.head()

KeyError: 'ville_align'

In [ ]:
#encore quelques médecins manquants à cause des CP
print( len(medecins_insee[medecins_insee["Code_commune_INSEE"].fillna("")==""]) )
print( len(medecins_insee[medecins_insee["Code_commune_INSEE"].fillna("")==""])/len(medecins_insee) )

In [ ]:
medecins_insee[medecins_insee["Code_commune_INSEE"].fillna("")!=""].to_csv( os.path.join(outdir,"medecins.csv") )

## On recommence ...

Avec les nouvelles communes, certains noms de communes semblent être indiqués dans le champs "Ligne 5"

In [ ]:
data = pd.read_csv("../data/laposte_hexasmal.csv", sep=";")

In [ ]:
def fusion(x):
    if pd.isnull(x['Ligne_5']):
        return x['Nom_commune']
    else:
        return x['Ligne_5']
data['Nom_commune']=data.apply(fusion,axis=1)
data=data[["Code_commune_INSEE","Nom_commune","Code_postal"]]
data.drop_duplicates(inplace=True)
data

In [ ]:
medecins_insee=pd.merge(data, medecins, how="right", right_on=["CP","Ville"], left_on=["Code_postal","Nom_commune"])[["Code_commune_INSEE","Ville","CP","Sexe","Profession"]]
medecins_insee.head()

In [ ]:
##même taille que la liste initial des médecins ...
print(len(medecins)) 
print(len(medecins_insee)) 

In [ ]:
#mais ... des correspondances n'ont pas fontionner
medecins_manquants=medecins_insee[medecins_insee["Code_commune_INSEE"].fillna("")==""]
medecins_manquants

In [ ]:
#utilisation d'une librairie permettant de faire une matching approximatif
import difflib

# Pour chaque ville, on cherche la correspondance la plus proche dans la base des communes disponibles !
## ATTENTION: cette étape est longue en calcul -> appliqué sur les unique()
def proceed(x):
    corresp = difflib.get_close_matches(x, data["Nom_commune"])
    if len(corresp)==0:
        return ""
    else:
        return corresp[0]

#rename=pd.Series(medecins_manquants ["Ville"].unique()).apply(proceed)

In [ ]:
corresp=pd.concat([pd.Series(medecins_manquants["Ville"].unique()),rename], axis=1)
corresp.rename(columns={0:"OldVille",1:"Ville"},inplace=True)
corresp

In [ ]:
len(corresp['OldVille'].unique())

In [ ]:
medecins_manquants_filled=pd.merge(medecins_manquants, corresp, left_on="Ville", right_on="OldVille", suffixes=("old",""))[["Ville","OldVille","CP","Sexe",'Profession']]
medecins_manquants_filled

In [ ]:
medecins_insee2=pd.merge(data, medecins_manquants_filled, how="right", left_on=["Code_postal","Nom_commune"], right_on=["CP","Ville"])[["Code_commune_INSEE","Ville","OldVille","CP","Sexe","Profession"]]
medecins_insee2.head()
len(medecins_insee2)

In [ ]:
#mais ... il en reste toujours sans correspondance !!
medecins_manquants2=medecins_insee2[medecins_insee2["Code_commune_INSEE"].fillna("")==""]
medecins_manquants2

In [ ]:
medecins_manquants_filled['dpt']=medecins_manquants2['CP']//1000
data['dpt']=data['Code_postal']//1000
medecins_insee2_villedpt=pd.merge(data[["Code_commune_INSEE",'dpt',"Nom_commune"]].drop_duplicates(), 
                                  medecins_manquants_filled, 
                                  how="right", left_on=['dpt',"Nom_commune"], right_on=['dpt',"Ville"])[["Code_commune_INSEE","Ville","OldVille","CP","Sexe","Profession"]]
len(medecins_insee2_ville)

In [ ]:
print(len(data[['dpt',"Nom_commune"]]))
print(len(data[["Code_commune_INSEE",'dpt',"Nom_commune"]].drop_duplicates()))
print(len(data.drop_duplicates()))
data

In [ ]:
medecins_insee2_ville=pd.merge(data, medecins_manquants_filled, how="right", left_on=["Nom_commune"], right_on=["CP","Ville"])[["Code_commune_INSEE","Ville","OldVille","CP","Sexe","Profession"]]
len(medecins_insee2_ville)

In [ ]:
medecins_manquants=medecins_insee_ville[medecins_insee_ville["Code_commune_INSEE"].fillna("")==""]
medecins_manquants

In [ ]:
np.sum( (medecins_insee_ville['CP']-medecins_insee_ville['Code_postal']) <=1000 )

In [ ]:
medecins_insee_ville[medecins_insee_ville['CP']-medecins_insee_ville['Code_postal']>1000]

## Nouveaux test avec une autre base !!

Base ODS [https://www.data.gouv.fr/fr/datasets/r/557f25da-4bfd-4306-a342-9bd2adb49ba9] datant de 2014